In [1]:
import ffmpeg
import gradio as gr

from pathlib import Path
from datetime import datetime
from projetMAB1.model import Model
from projetMAB1.indexer import index

ModuleNotFoundError: No module named 'projetMAB1'

In [2]:
VIDEO_FOLDER_PATH = "../data/runtime/videos/"
EXPORT_FOLDER_PATH = "../data/runtime/export/"
model = Model()

NameError: name 'Model' is not defined

In [ ]:
def query(text, k):
    matches = model.query(text, k=k)
    return sorted(
        [*[(path, '{0:.2f}'.format(similarity)) for path, similarity in matches]],
        key = lambda x: x[1], reverse = True
    ), [path for path, _ in matches]

In [ ]:
videos = sorted(Path("../data/runtime/videos/").glob("*.mp4"))
images = sorted(Path("../data/runtime/frames/").glob("*.mp4"))
images_by_video = {}
video_by_images = {}

for image in images:

    key = image.stem.split("_")[0]
    if key not in images_by_video:
        images_by_video[key] = [image]
    else:
        images_by_video[key].append(image)

    video_by_images[image] = key

In [ ]:
def export(gallery, evt: gr.SelectData):
    video_name, start_frame, end_frame = Path(gallery[evt.index]["name"]).stem.split("_")
    in_file = ffmpeg.input(f"{VIDEO_FOLDER_PATH}{video_name}.mp4", loglevel='warning', copyts=None)
    out_file = f"{EXPORT_FOLDER_PATH}{datetime.now()}.mp4"
    (
        ffmpeg.concat(in_file.trim(start_frame=start_frame, end_frame=end_frame),
                      )
        .filter('stepts', expr='PTS-STARTPTS')
        .ouput(out_file).run()
    )

In [ ]:
examples = [
    ["A tennis match"],
	["A footbal match"],
	["A baseball match"],
	["Cooking"],
	["A video game"],
	["A political man"],
	["A political debate"],
	["Barack Obama"],
	["Abraham Lincoln"],
	["A concert"],
	["A computer screen"],
	["Advertising"],
	["Dragon ball"],
	["A pair of shoes"],
	["Elephants"],
	["An iPhone"],
	["A video clip"],
	["A drawing"],
	["A film"],
	["A makeup tutorial"],
	["Industrial machinery"],
	["A person repairing a car"]
]

In [ ]:
css = "footer {display:none !important;} .gradio-container {min-height: 0px !important;}"
with gr.Blocks(css=css) as demo:

    gr.Markdown("# Video scene retrieval")

    with gr.Tab("Query sources"):
        text_input = gr.Textbox()
        k = gr.Slider(1, 10, 3, step=1, label="Number of results")
        examples = gr.Examples(examples, text_input)
        query_button = gr.Button("Query")
        matches = gr.Dataset(components=["image", "text"], label="Matches", headers=["Matching frame", "Similarity"])

    with gr.Tab("Export"):
        export_gallery = gr.Gallery()

    with gr.Tab("Add sources"):
        video_to_index = gr.Video()
        progress = gr.Progress()
        index_button = gr.Button("Index video")

    with gr.Tab("View Index"):
        for video in videos:
            with gr.Row():
                with gr.Column():
                    gr.Video(str(video), label=video.stem)
                with gr.Column():
                    ds = gr.Dataset(
                        components=[gr.Image(visible=False)],
                        label=video.name,
                        samples=[[x] for x in images_by_video[video.stem]]
                    )

    index_button.click(index, inputs=video_to_index)
    query_button.click(query, inputs=[text_input, k], outputs=[matches, export_gallery])
    text_input.submit(query, inputs=[text_input, k], outputs=[matches, export_gallery])
    export_gallery.select(export, inputs=export_gallery)

demo.lunch()
        